In [21]:
import numpy as np
import pandas as pd

In [74]:
#ПРИХИ

#загружаем две таблицы с приходами по всем ТГ 4001 и 4093
opt_prih = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\4093.xlsx',
    usecols=[0,4,9,11,12,15],names=['Производитель','Тип 1','Номер','Приходы_4093','ТПГ','ОПТ'])

cs_prih = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\4001.xlsx',
    usecols=[0,4,9,11,12],names=['Производитель','Тип 1','Номер','Приходы_4001','ТПГ'])

# убираем итог, оптовый товар и нулевые приходы
opt_prih.drop(index=len(opt_prih.Производитель.tolist())-1,inplace=True)
cs_prih.drop(index=len(cs_prih.Производитель.tolist())-1,inplace=True)

opt_prih = opt_prih[(opt_prih['ОПТ'] == 'Нет') & (opt_prih['Приходы_4093'] != 0)]
del opt_prih['ОПТ']
cs_prih = cs_prih[ (cs_prih['Приходы_4001'] != 0)]

#соединием 2 таблицы с приходами 4001 и 4093 в одну и делим на иф и бк
prih_merged = pd.merge(opt_prih,cs_prih,how = 'outer')
prih_merged.fillna(value=0,inplace=True)


prih_if = prih_merged[prih_merged['ТПГ']< 1499]
prih_bk = prih_merged[(prih_merged['ТПГ'] > 1499) & 
                      (prih_merged['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')& 
                      (prih_merged['Тип 1'] != 'СЕРЬГИ-КОНГО')]
del prih_if['ТПГ'], prih_bk['ТПГ']


In [23]:
#КОЛЬЦА ИФ

#загружаем две таблицы остатки КОЛЬЦА 4001 и 4093 и одну продажи
balance=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_к',skiprows=9) #использовать sheet_name='ост' для неколец
balance_4093=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_4093',skiprows=9,names=['Номер','Остаток_4093'])
sales=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12)

#соединяем продажи, остатки 4093 и остатки в одну таблицу
merged=pd.merge(balance,balance_4093,on='Номер',how = 'left')
del sales['Тип 1'],sales['Производитель']
merged=pd.merge(merged,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))

# фильтр приходов по кольцам
prih_if = prih_if[
    (prih_if['Тип 1'] == 'КОЛЬЦО')|
    (prih_if['Тип 1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ')|
    (prih_if['Тип 1'] == 'КОЛЬЦО ПЕЧАТКА')
]

#соединяем приходы, остатки и продажи
merged=pd.merge(prih_if,merged,how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
del items['Описание 3']

final=pd.merge(merged,items,on='Номер',how = 'left')

#удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

col_names=final.columns.tolist()

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
#final['сумма']=final.groupby(by='Артикул товара')[col_names[5:7]].sum(axis=1)

#выстраиваем столбцы в нужном порядке
columns=[col_names[18],col_names[1],col_names[0]]+col_names[19:21]+[col_names[2]]+col_names[21:25]+col_names[12:18]+col_names[5:12]+col_names[3:5]+col_names[-5:]
final=final[columns]
#final=final.set_index('Артикул товара',drop=True, inplace=True)

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_кольца.xlsx')

In [49]:
#НЕКОЛЬЦА ИФ

#загружаем две таблицы остатки КОЛЬЦА 4001 и 4093 и одну продажи
balance=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост',skiprows=9) #использовать sheet_name='ост' для неколец
balance_4093=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_4093',skiprows=9,names=['Номер','Остаток_4093'])
sales=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12)

#СОЕДИНЯЕМ ПРОДАЖИ, ОСТАТКИ 4093 И ОСТАТКИ В ОДНУ ТАБЛИЦУ
merged = pd.merge(balance,balance_4093,on='Номер',how = 'left')
del sales['Тип 1'],sales['Производитель']
merged = pd.merge(merged,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))
# фильтр приходов по кольцам
prih_if = prih_if[
    (prih_if['Тип 1'] != 'КОЛЬЦО')&
    (prih_if['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')&
    (prih_if['Тип 1'] != 'КОЛЬЦО ПЕЧАТКА')
]

#соединяем приходы, остатки и продажи
merged=pd.merge(prih_if,merged,how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
del items['Размер']

final=pd.merge(merged,items,on='Номер',how = 'left')

#удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]

#выстраиваем столбцы в нужном порядке
col_names = final.columns.tolist()
columns=[col_names[18],col_names[1],col_names[0]]+col_names[19:21]+[col_names[2]]+col_names[21:25]+col_names[12:18]+col_names[5:12]+col_names[3:5]+col_names[-5:]
final = final[columns]

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
final['сумма'] = final[col_names[3:18]].sum(axis=1)
final=final[final['сумма'] >0]
del final['сумма']

#Удаляем буквы и зодиаки
final = final[(final['Дизайн'] !='ИФ БУКВЫ') & (final['Дизайн'] !='ИФ ЗОДИАК')]

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_некольца.xlsx')


In [75]:
#БК МОЖНО ОТКЛЮЧИТЬ НЕ нужный тип1 в конце

#загружаем две таблицы остатки 4001 и 4093 и продажи
balance=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='ост',skiprows=9)

balance_4093=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='ост_4093',skiprows=9,names=['Номер','Остаток_4093'])

sales=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='прод',skiprows=12)

#соединяем продажи, остатки 4093 и остатки в одну таблицу
merged = pd.merge(balance,balance_4093,on='Номер',how = 'left')
del sales['Тип 1'],sales['Производитель']
merged = pd.merge(merged,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))


#соединяем приходы, остатки и продажи
merged = pd.merge(prih_bk,merged,how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='карточки',skiprows=6)

final = pd.merge(merged,items,on='Номер',how = 'left')

#удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

col_names=final.columns.tolist()

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
#final['сумма']=final.groupby(by='Артикул товара')[col_names[5:7]].sum(axis=1)

#выстраиваем столбцы в нужном порядке
columns=[col_names[18],col_names[1],col_names[0]]+col_names[19:21]+[col_names[2]]+col_names[21:25]+col_names[12:18]+col_names[5:12]+col_names[4:2:-1]+col_names[-4:]
final=final[columns]

#final=final.set_index('Артикул товара',drop=True, inplace=True)

#делим на кольца и некольца в кольцах убираем описание 3 и суммируем приходы 4001+4093 в "приходы". 
#в некольцах убираем размер и буквы с зодиаками

final_ring = final[final['Тип 1'].str.contains('КОЛЬЦО')].drop(['Описание 3','Остаток_4093'], axis=1)
final_ring['Приходы'] = final_ring['Приходы_4001']+final_ring['Приходы_4093']
del final_ring['Приходы_4001'], final_ring['Приходы_4093']
ring_cols = final_ring.columns.tolist()
final_ring = final_ring[ring_cols[:21]+[ring_cols[-1]]+ring_cols[21:25]]

final_other = final[~final['Тип 1'].str.contains('КОЛЬЦО')].drop(['Размер'], axis=1)

final_other = final_other[~((final_other['Тип 1']== 'ПОДВЕС ДЕКОРАТИВНЫЙ') & 
                            (final_other['Дизайн'] == 'ИФ ДВУСПЛАВ ДЕКОР') & 
                            (final_other['Дизайн'] =='ИФ БУКВЫ') & 
                            (final_other['Дизайн'] =='ИФ ЗОДИАК'))]

# сохраняем кольца и некольца на отдельные вкладки
with pd.ExcelWriter (
     r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\заказы_бк.xlsx'
) as writer:
    final_ring.to_excel(writer, sheet_name = 'кольца')
    final_other.to_excel(writer, sheet_name = 'некольца')


In [72]:
prih_bk[(prih_bk['Тип 1']=='КОЛЬЦО')]
final_ring['Приходы'] = final_ring['Приходы_4001']+final_ring['Приходы_4093']

,Производитель,Тип 1,Номер,Приходы_4093,Приходы_4001
149,ИП Стишков М.А.,КОЛЬЦО,ТОВ0577902,NaN,5.0
150,ИП Стишков М.А.,КОЛЬЦО,ТОВ0577905,NaN,11.0
151,ИП Стишков М.А.,КОЛЬЦО,ТОВ0577952,NaN,1.0
152,ИП Стишков М.А.,КОЛЬЦО,ТОВ0577954,NaN,2.0
153,ИП Стишков М.А.,КОЛЬЦО,ТОВ0577956,NaN,3.0
161,ИП Стишков М.А.,КОЛЬЦО,ТОВ0588112,NaN,4.0
162,ИП Стишков М.А.,КОЛЬЦО,ТОВ0588115,NaN,9.0
1428,"ООО ""КЮЗ ""Золотые традиции""",КОЛЬЦО,ТОВ1079067,NaN,4.0
1429,"ООО ""КЮЗ ""Золотые традиции""",КОЛЬЦО,ТОВ1079068,NaN,4.0
1430,"ООО ""КЮЗ ""Золотые традиции""",КОЛЬЦО,ТОВ1079075,NaN,1.0
